In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
driver = webdriver.Chrome()
driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
time.sleep(3)


In [7]:
#예외 처리
CAFE_NAME = 'rocksoccer'
BOARD_NAME = 'ADs2'
QUERY = '쿠플'
driver.get(f'http://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search?r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&query={QUERY}')
time.sleep(3)
import pandas as pd
from bs4 import BeautifulSoup

# 본문과 댓글을 위한 별도의 DataFrame 초기화
df_columns_posts = ['num', 'subject', 'nick', 'write_time', 'views', 'url', 'contents']
df_posts = pd.DataFrame(columns=df_columns_posts)

df_columns_comments = ['cmt_num', 'cmt_writer', 'cmt_time', 'cmt_txt']
df_comments = pd.DataFrame(columns=df_columns_comments)

# csv파일 읽기
urls_df = pd.read_csv('ROCKSOCCER_쿠플_23pages_inner_links.csv', header=None, names=['url_list'])
urls = ["http://m.cafe.daum.net" + url if not url.startswith("http://") and not url.startswith("https://") else url for url in urls_df['url_list'].tolist()]

for url in urls:
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    inp_num = url.split('/')[-1].split('?')[0]
    
    subject = soup.body.find('h3', class_='tit_subject')
    # 게시물 정보 추가 (게시물 본문 크롤링 부분에 추가)
    if subject is None:
        print(url, '지워진 게시물입니다.')
        continue
    else:
        subject = subject.get_text(strip=True)
        if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
            nick = soup.body.find('span', class_='sr_only').next_sibling
        else:
            nick = ''
        num_subject = soup.body.find_all('span', class_='num_subject')
        write_time = num_subject[0].get_text()
        views = num_subject[1].get_text()
        contents = soup.body.find('div', id='article').get_text('\n', strip=True)
        
        # 게시물 정보를 df_posts에 추가
        new_row_post = {'num': inp_num, 'subject': subject, 'nick': nick, 'write_time': write_time, 'views': views, 'url': url, 'contents': contents}
        df_posts = pd.concat([df_posts, pd.DataFrame([new_row_post])], ignore_index=True)
    
    
        # 댓글
        cmt_total = soup.body.find('article', id='mArticle').find('span', class_="num_total").get_text()
        driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments'.format(CAFE_NAME, BOARD_NAME, inp_num))
        time.sleep(1)
        # 페이지 숫자를 확인하기 위해 캡쳐
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 페이지 숫자 확인
        paging_nav = soup.body.find('span', id="pagingNav")
        if paging_nav is None:
            cmt_page_max = 0
        else:
            num_pages = paging_nav.find_all('span', class_="num_page")
            if len(num_pages) == 0:
                cmt_page_max = 0
            else:
                cmt_page_max = int(num_pages.pop().get_text())
        
        # 댓글 정보 추가 (댓글 크롤링 부분에 추가)
        for i in range(cmt_page_max):
            j = cmt_page_max - i
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            cmt_all = soup.body.find('ul', id="commentList").find_all('li')
            for k in cmt_all:
                # 기본값을 설정합니다
                cmt_reply = False
                reply_prefix = ''
                
                if k.div.span.find_all('span', class_="txt_bar"):
                    if k['class'] == ['reply_on']:
                        cmt_reply = True
                        reply_prefix = 'ㄴ'
                
                # 삭제된 댓글 처리
                if k.find('strong', class_='tit_info'):
                    if k.find('strong', class_='tit_info').find('span', class_='txt_detail').get_text(strip=True) == '삭제된 댓글입니다.':
                        continue
                
                cmt_num = inp_num + '_' + k['id'].split('comment_')[1]
                cmt_writer_elem = k.div.find('span', class_='username')
                if cmt_writer_elem:
                    cmt_writer = cmt_writer_elem.get_text(strip=True)
                else:
                    continue  # 삭제된 댓글이므로 건너뜁니다.
                    
                cmt_time = k.div.span.find('span', class_="created_at").get_text()
                cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                
                # 댓글 정보를 df_comments에 추가
                new_row_comment = {'cmt_num': reply_prefix + cmt_num, 'cmt_writer': cmt_writer, 'cmt_time': cmt_time, 'cmt_txt': cmt_txt}
                df_comments = pd.concat([df_comments, pd.DataFrame([new_row_comment])], ignore_index=True)
            
            if j > 1:
                driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page={3}&mode=regular&cdepth={4}&page={5}'.format(CAFE_NAME, BOARD_NAME, inp_num, j, '0002100000', (j-1)))
                time.sleep(2)

df_posts.to_csv('RS_쿠플_posts.csv', index=False)
df_comments.to_csv('RS_쿠플_comments.csv', index=False)

driver.close()


In [8]:
df_posts

,num,subject,nick,write_time,views,url,contents
0,489458,그나저나 정말 쿠플은 멀티뷰 해줄 생각이 없는 걸까요,잔소리 봇,24.05.19,282,http://m.cafe.daum.net/rocksoccer/ADs2/489458?...,생각해보니 티빙도 멀티뷰는 안되긴 하네요\n근데 진짜 여러 프로리그 중계가 목표라면...
1,489439,아이폰으로 쿠플 보는데 왜 자꾸 가로고정 풀리죠!,Clarence Seedorf,24.05.19,105,http://m.cafe.daum.net/rocksoccer/ADs2/489439?...,? 아무것도 누르지 않았는데 자꾸 풀려서 세로모드로 바뀌네요\n저만 이러나요
2,488900,쿠플이 EPL까지 먹으면 축구 천하 통일한거죠?,축구부장,24.05.19,323,http://m.cafe.daum.net/rocksoccer/ADs2/488900?...,아 아직 통일 못한 챔스가 남아있나
3,488895,쿠플은 이승모 얘기 아직 안 들어갔나보다 ㄷㄷ,버츠비,24.05.19,141,http://m.cafe.daum.net/rocksoccer/ADs2/488895?...,배성재랑 현영민\n직접 경기보고 이승모로 바뀐거 알아챔 ㄷㄷㄷㄷ
4,488622,[움짤]정유미84가 짚어보는 지지대더비 관전 포인트.gif,음교수2,24.05.18,"2,873",http://m.cafe.daum.net/rocksoccer/ADs2/488622?...,쿠플픽 뭐하냐 빨리 섭외하라고....
...,...,...,...,...,...,...,...
453,307095,쿠팡 4990원,스마일리,22.04.12,"1,780",http://m.cafe.daum.net/rocksoccer/ADs2/307095?...,로켓배송 쓰면서 쿠팡플레이까지 개꿀아닙니까??\n2900원에 가입했었믄데 4990원...
454,307093,왓챠나 웨이브였으면 그래도 긍정적으로 봤을텐데,나상호,22.04.12,171,http://m.cafe.daum.net/rocksoccer/ADs2/307093?...,쿠플이라니 솔직히 기대보다 걱정이 크긴하네요.. 슬슬 구독료도 올릴텐데 컨텐츠 면에...
455,307080,사실상 온라인 중계 유료화죠?,그리핀도르,22.04.12,843,http://m.cafe.daum.net/rocksoccer/ADs2/307080?...,쿠플이 k리그만 따로 무료중계 해줄거 같지않고\n5900원인가? 이거내야 볼수 있...
456,292693,? 국축보는데 K리그 어플 안 씁니까?,UAENA,22.03.10,"1,090",http://m.cafe.daum.net/rocksoccer/ADs2/292693?...,인스타 - 구단 인스타\n다운로더 - 인스타 사진 원본으로 다운\n쿠플 - 국대 경...


In [9]:
df_comments

,cmt_num,cmt_writer,cmt_time,cmt_txt
0,489458_1,No.30 Messi,24.05.19,아이폰 유저인데 화면 창모드에서 다시 전체화면으로 바꾸면 무조건 일시정지 되는것도 ...
1,489458_3,LeBron James,24.05.19,22222
2,489458_2,윤비트,24.05.19,전 전체화면 할 때마다채팅창 나오는게 기본인것 좀 바꿔줬으면..
3,489439_1,포항스틸러스!,24.05.19,고정해놓으셨나요?
4,489439_2,Clarence Seedorf작성자 본인 여부작성자,24.05.19,네
...,...,...,...,...
1846,278531_14,범!!! 내려온다!!,22.01.13,라이브 중에도 말했지만 쿠팡이 정식적으로 돈주고 사는게 아니면 반대입니다...또 잘...
1847,278531_16,Mr.Moon,22.01.13,"그럴일이 없을듯.경기하는것도 연맹이랑 협상하면초청 선수단 관리,경기장,훈련장 섭외,..."
1848,278531_17,이날치,22.01.13,스폰서 가좋을듯
1849,278531_19,나상호,22.01.13,쿠팡 들어오면 뉴미디어 유료화로 갈거고 그럼 반대.. 아직은 유료화 할수있는 시장이...
